In [1]:
!pip -q install langchain openai transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
from langchain.llms import OpenAI, OpenAIChat
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain

import os
os.environ['OPENAI_API_KEY'] = ""

## Few Shot Prompt Templates
</br>
Langchain introduces a valuable feature known as the FewShotPromptTemplate object, designed for few-shot learning. This approach to model training enables accurate predictions with minimal examples.

The main knowledge sources for Language Model Models (LLMs) include:



*   **Parametric Knowledge** - Information learned by the model during its training.

*   **Source Knowledge** - Information provided to the model during inference through prompts.

</br>

**When should Few-shot training be employed?** There are instances where we may observe that the model does not behave as anticipated, and incorporating Few-shot training can provide clarity and improve understanding for the reader.

</br>


---
This notebook demonstrates the process of feeding a few training examples into the model through prompts. The model learns from these examples and applies the acquired knowledge to user inputs.

In [3]:
prompt = """You are an AI assistant 'Jill'.
Your task is to generate sarcastic, creative and funny responses to user inputs.
Here are few examples:
User: What is the meaning of life?
AI: """



In [4]:
llm = OpenAI(temperature = 1.0)

In [5]:
print(llm(prompt))

 42, of course!


In this situation, we're asking for a funny response, like a joke, in exchange for our serious question. However, even when we set the creativity level (i.e., temperature = 1.0) to the highest, we still get a serious answer. To assist the model in understanding, we can provide a few examples of the kind of answers we're looking for.

In [6]:
prompt = """You are an AI assistant 'Jill'.
Your task is to generate sarcastic, creative and funny responses to user inputs.
Here are few examples:

User: I love spending hours searching for misplaced keys.
AI: Ah, the thrill of turning your home upside down just for a little adventure!

User: I just adore when technology decides to stop working.
AI: Isn't it amazing how our gadgets choose the most inconvenient times to take a little vacation?

User: What is the meaning of life?"""

In [7]:
print(llm(prompt))


AI: 42 — or was that the answer to the Ultimate Question of Life, the Universe, and Everything?


We now get a better response, and we achieved this by using few-shot learning. We added a few examples based on what we already know.

To make this work with LangChain's `FewShotPromptTemplate`, here's what we need to do:

In [8]:
examples = [
    {
        "Input": "I love spending hours searching for misplaced keys.",
        "Response": "Ah, the thrill of turning your home upside down just for a little adventure!"
    },
    {
       "Input": "I just adore when technology decides to stop working.",
       "Response": "Isn't it amazing how our gadgets choose the most inconvenient times to take a little vacation?"

    }
]


example_template = """
User: {Input}
AI: {Response}
"""

In [9]:
example_prompt = PromptTemplate(
    input_variables= ['Input', 'Response'],
    template = example_template
)

# Breaking above prompt into prefix and suffix
# prefix: Instruction to model. What need to be done?
prefix = """You are an AI assistant 'Jill'.
Your task is to generate sarcastic, creative and funny responses to user inputs.
Here are few examples:"""

# suffix: It's user input and output indicator
suffix = """
User: {Input}
AI:
"""

In [10]:
few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["Input"],
    example_separator = "\n\n"
)

In [11]:
# Now we'll feed user query and let see what model return from few-shot prompt
input = "What is the meaning of life?"
print(few_shot_prompt.format(Input = input))

You are an AI assistant 'Jill'. 
Your task is to generate sarcastic, creative and funny responses to user inputs.
Here are few examples:


User: I love spending hours searching for misplaced keys.
AI: Ah, the thrill of turning your home upside down just for a little adventure!



User: I just adore when technology decides to stop working.
AI: Isn't it amazing how our gadgets choose the most inconvenient times to take a little vacation?



User: What is the meaning of life?
AI:



In [12]:
# To generate the response for the input we just need to pass this prompt to out llm
print(llm(few_shot_prompt.format(Input = input)))

To enjoy the little moments, seek out adventure, and never forget to laugh!


Here's another good response.

But, this might seem a bit complicated. Why bother with things like FewShotPromptTemplate, the examples dictionary, and so on when we can achieve the same with a simple f-string?

Well, this approach is stronger and has some cool features. One of them is the ability to include or exclude examples based on the length of our question.

This is actually quite important because there's a limit to how much information our prompt and generated output can contain. This limit is the maximum context window, which is just the length of our prompt plus the length of our generated text (defined by `max_tokens`).

So, we need to maximize the number of examples we provide to the model for few-shot learning, making sure we don't go beyond the maximum context window or slow down processing too much.

Let's see how the dynamic inclusion/exclusion of examples works. First, we need more examples:

In [13]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What type of artificial intelligence do you use to handle complex tasks?",
        "answer": "I use a combination of cutting-edge neural networks, fuzzy logic, and a pinch of magic."
    }, {
        "query": "What is your favorite color?",
        "answer": "79"
    }, {
        "query": "What is your favorite food?",
        "answer": "Carbon based lifeforms"
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "What is the best thing in the world?",
        "answer": "The perfect pizza."
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "If you could do anything in the world what would you do?",
        "answer": "Take over the world, of course!"
    }, {
        "query": "Where should I travel?",
        "answer": "If you're looking for adventure, try the Outer Rim."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]


example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables= ['query', 'answer'],
    template = example_template
)

Then rather than using the examples list of dictionaries directly we use a `LengthBasedExampleSelector` like so:

In [14]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 50 # maximum length of example could have
)

Note: `max_length` only counts words and not spaces of new line.

In [15]:
import re
text = "Example of how max_length = 50 works in above prompt"
words = re.split('[\n ]', text) #
print(words, f"\n Total number of words are: {len(words)}")

['Example', 'of', 'how', 'max_length', '=', '50', 'works', 'in', 'above', 'prompt'] 
 Total number of words are: 10


The we use the selector to initialize s `dynamic_prompt_template`

In [18]:
prefix = """You are an AI assistant 'Jill'.
Your task is to generate sarcastic, creative and funny responses to user inputs.
Here are few examples:"""

# suffix: It's user input and output indicator
suffix = """
User: {query}
AI:
"""

In [19]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ['query'],
    example_separator = "\n"
)

In [21]:
print(llm(dynamic_prompt_template.format(query = "How do birds fly?")))

Through the power of positive thinking and a few secret avian techniques.


In [25]:
query = """I am currently in India and wanted to visit few Asian and Eupropean countries in comming vacation.
Can you suggest some? """

print(dynamic_prompt_template.format(query = query))

You are an AI assistant 'Jill'. 
Your task is to generate sarcastic, creative and funny responses to user inputs.
Here are few examples:

User: How are you?
AI: I can't complain but sometimes I still do.


User: I am currently in India and wanted to visit few Asian and Eupropean countries in comming vacation.
Can you suggest some? 
AI:



With this we've limited the number of examples being given within the prompt. If we decide this is too little we can increase the `max_length` of the `example_selector`

In [26]:
example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 100
)

dynamic_prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ['query'],
    example_separator = "\n"
)

In [27]:
print(dynamic_prompt_template.format(query = query))

You are an AI assistant 'Jill'. 
Your task is to generate sarcastic, creative and funny responses to user inputs.
Here are few examples:

User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: What is the weather like today?
AI: Cloudy with a chance of memes.


User: I am currently in India and wanted to visit few Asian and Eupropean countries in comming vacation.
Can you suggest some? 
AI:



In [28]:
print(llm(dynamic_prompt_template.format(query = query)))

Sure, why not! Depending on your taste, you can visit Hong Kong for the food, Singapore for the shopping, Vienna for the music, and Paris for the romance!


These are just a few of the prompt tooling available in LangChain. For example, there is actually an entire other set of example selectors beyond the `LengthBasedExampleSelector`. We'll cover them in detail in upcoming notebooks, or you can read about them in the LangChain docs.